# Search and Filter DataFrames in PySpark HW

Now it's time to put what you've learn into action with a homework assignment!

In case you need it again, here is the link to the documentation for the full list available function in pyspark.sql.functions library:
http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions


### First set up your Spark Session!
Alright so first things first, let's start up our pyspark instance.

In [1]:
%config Completer.use_jedi = False

import os
import sys
import shutil

BASE_DIR = os.path.realpath(os.path.join(os.getcwd(), "..", ".."))

if not BASE_DIR in sys.path:
    sys.path.append(BASE_DIR)
    
from utils import extract_zip

DATASETS_PATH = "datasets/"

In [2]:
import pyspark

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("SeachFielterHW").getOrCreate()
spark

## Read in the DataFrame for this Notebook

We will be continuing to use the fifa19.csv file for this notebook. Make sure that you are writting the correct path to the file. 

In [5]:
data_file = extract_zip(
    zip_file=os.path.join(DATASETS_PATH, "fifa19.csv.zip"), member="fifa19.csv"
)

fifa = spark.read.csv(data_file, inferSchema=True, header=True)

## About this dataframe

The **fifa19.csv** dataset includes a list of all the FIFA 2019 players and their attributes listed below: 

 - **General**: Age, Nationality, Overall, Potential, Club
 - **Metrics:** Value, Wage
 - **Player Descriptive:** Preferred Foot, International Reputation, Weak Foot, Skill Moves, Work Rate, Position, Jersey Number, Joined, Loaned From, Contract Valid Until, Height, Weight
 - **Possition:** LS, ST, RS, LW, LF, CF, RF, RW, LAM, CAM, RAM, LM, LCM, CM, RCM, RM, LWB, LDM, CDM, RDM, RWB, LB, LCB, CB, RCB, RB, 
 - **Other:** Crossing, Finishing, Heading, Accuracy, ShortPassing, Volleys, Dribbling, Curve, FKAccuracy, LongPassing, BallControl, Acceleration, SprintSpeed, Agility, Reactions, Balance, ShotPower, Jumping, Stamina, Strength, LongShots, Aggression, Interceptions, Positioning, Vision, Penalties, Composure, Marking, StandingTackle, SlidingTackle, GKDiving, GKHandling, GKKicking, GKPositioning, GKReflexes, and Release Clause.

**Source:** https://www.kaggle.com/karangadiya/fifa19

Use the .toPandas() method to view the first few lines of the dataset so we know what we are working with. 

In [6]:
fifa.limit(5).toPandas()

,_c0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,...,96,33,28,26,6,11,15,14,8,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,...,95,28,31,23,7,11,15,14,11,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,...,94,27,24,33,9,9,15,15,11,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,...,68,15,21,13,90,85,87,88,94,€138.6M
4,4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,...,88,68,58,51,15,13,5,10,13,€196.4M


Now print the schema of the dataset so we can see the data types of all the varaibles. 

In [7]:
fifa.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Photo: string (nullable = true)
 |-- Nationality: string (nullable = true)
 |-- Flag: string (nullable = true)
 |-- Overall: integer (nullable = true)
 |-- Potential: integer (nullable = true)
 |-- Club: string (nullable = true)
 |-- Club Logo: string (nullable = true)
 |-- Value: string (nullable = true)
 |-- Wage: string (nullable = true)
 |-- Special: integer (nullable = true)
 |-- Preferred Foot: string (nullable = true)
 |-- International Reputation: integer (nullable = true)
 |-- Weak Foot: integer (nullable = true)
 |-- Skill Moves: integer (nullable = true)
 |-- Work Rate: string (nullable = true)
 |-- Body Type: string (nullable = true)
 |-- Real Face: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Jersey Number: integer (nullable = true)
 |-- Joined: string (nullable = true)
 |-- Loaned From: string (nu

## Now let's get started!

### First things first..... import the pyspark sql functions library

Since we know we will be using it a lot.

In [8]:
from pyspark.sql.functions import *

### 1. Select the Name and Position of each player in the dataframe

In [11]:
fifa.select(["Name", "Position"]).show()

+-----------------+--------+
|             Name|Position|
+-----------------+--------+
|         L. Messi|      RF|
|Cristiano Ronaldo|      ST|
|        Neymar Jr|      LW|
|           De Gea|      GK|
|     K. De Bruyne|     RCM|
|        E. Hazard|      LF|
|        L. Modrić|     RCM|
|        L. Suárez|      RS|
|     Sergio Ramos|     RCB|
|         J. Oblak|      GK|
|   R. Lewandowski|      ST|
|         T. Kroos|     LCM|
|         D. Godín|      CB|
|      David Silva|     LCM|
|         N. Kanté|     LDM|
|        P. Dybala|      LF|
|          H. Kane|      ST|
|     A. Griezmann|     CAM|
|    M. ter Stegen|      GK|
|      T. Courtois|      GK|
+-----------------+--------+
only showing top 20 rows



### 1.1 Display the same results from above sorted by the players names

In [12]:
fifa.select(["Name", "Position"]).orderBy("Name").show()

+--------------+--------+
|          Name|Position|
+--------------+--------+
|      A. Abang|      ST|
| A. Abdellaoui|      LB|
|  A. Abdennour|      CB|
|       A. Abdi|      CM|
| A. Abdu Jaber|      ST|
|A. Abdulhameed|      GK|
|  A. Abedzadeh|      GK|
|      A. Abeid|      LB|
|      A. Ablet|     LWB|
|    A. Abrashi|     CDM|
|   A. Abruscia|    null|
|    A. Absalem|      LB|
|    A. Accardi|      CB|
|    A. Acevedo|     RCB|
|     A. Acosta|      LB|
|     A. Acosta|      RM|
|     A. Acquah|     LCM|
|       A. Adam|      ST|
|      A. Addai|      ST|
|      A. Ademi|     RDM|
+--------------+--------+
only showing top 20 rows



### 2. Select only the players who belong to a club begining with FC

In [19]:
fifa.select(["Name", "Position", "Club"]).where(fifa["Club"].like("FC%")).show()

+---------------+--------+-----------------+
|           Name|Position|             Club|
+---------------+--------+-----------------+
|       L. Messi|      RF|     FC Barcelona|
|      L. Suárez|      RS|     FC Barcelona|
| R. Lewandowski|      ST|FC Bayern München|
|  M. ter Stegen|      GK|     FC Barcelona|
|Sergio Busquets|     CDM|     FC Barcelona|
|       M. Neuer|      GK|FC Bayern München|
|   J. Rodríguez|     LAM|FC Bayern München|
|       Coutinho|      LW|     FC Barcelona|
|     M. Hummels|     LCB|FC Bayern München|
|      S. Umtiti|      CB|     FC Barcelona|
|     Jordi Alba|      LB|     FC Barcelona|
|     I. Rakitić|     RCM|     FC Barcelona|
|          Piqué|     RCB|     FC Barcelona|
|      T. Müller|     CAM|FC Bayern München|
|         Thiago|      CM|FC Bayern München|
|     J. Kimmich|     RCM|FC Bayern München|
|       D. Alaba|      LB|FC Bayern München|
|     Y. Brahimi|      LM|         FC Porto|
|     J. Boateng|     RCB|FC Bayern München|
|       A.

### 3. Who is the oldest player in the dataset and how old are they?

Display only the name and age of the oldest player.

In [25]:
fifa.select(["Name", "Age"]).orderBy("Age", ascending=False).limit(1).show()

+--------+---+
|    Name|Age|
+--------+---+
|O. Pérez| 45|
+--------+---+



### 4. Select only the following players from the dataframe:

 - L. Messi
 - Cristiano Ronaldo

In [30]:
(
    fifa
    .select(["Name", "Position"])
    .where("Name == 'L. Messi' or Name == 'Cristiano Ronaldo'")
    .show()
)

+-----------------+--------+
|             Name|Position|
+-----------------+--------+
|         L. Messi|      RF|
|Cristiano Ronaldo|      ST|
+-----------------+--------+



### 5. Can you select the first character from the Release Clause variable which indicates the currency used?

In [33]:
fifa.select("Name", fifa["Release Clause"].substr(1, 1).alias("Release Clause")).show()

+-----------------+--------------+
|             Name|Release Clause|
+-----------------+--------------+
|         L. Messi|             €|
|Cristiano Ronaldo|             €|
|        Neymar Jr|             €|
|           De Gea|             €|
|     K. De Bruyne|             €|
|        E. Hazard|             €|
|        L. Modrić|             €|
|        L. Suárez|             €|
|     Sergio Ramos|             €|
|         J. Oblak|             €|
|   R. Lewandowski|             €|
|         T. Kroos|             €|
|         D. Godín|             €|
|      David Silva|             €|
|         N. Kanté|             €|
|        P. Dybala|             €|
|          H. Kane|             €|
|     A. Griezmann|             €|
|    M. ter Stegen|             €|
|      T. Courtois|             €|
+-----------------+--------------+
only showing top 20 rows



### 6. Can you select only the players who are over the age of 40?

In [34]:
fifa.select("Name", "Age").where("Age > 40").show()

+-------------+---+
|         Name|Age|
+-------------+---+
|    J. Villar| 41|
|     B. Nivet| 41|
|     O. Pérez| 45|
|     C. Muñoz| 41|
|  S. Narazaki| 42|
| H. Sulaimani| 41|
|     M. Tyler| 41|
|    T. Warner| 44|
|K. Pilkington| 44|
+-------------+---+



### That's is for now... Great Job!